In [3]:
import numpy as np
import rasterio as rastr
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst
from osgeo import gdal

In [7]:
import json

# Load the GeoJSON dataset
with open('../Lab_07/spatial_data/field_polygons.geojson', 'r') as f:
    data = json.load(f)

# Transform the "Name" property
for feature in data['features']:
    index = feature['properties']['id'] - 1  # Subtract 1 to convert 1-based index to 0-based index
    name = feature['properties']['Name']
    feature['properties']['Name'] = f'{name}_{index}'

# Save the updated GeoJSON dataset
with open('fields_updated.geojson', 'w') as f:
    json.dump(data, f)


In [14]:
import glob

# Specify the folder path and the search pattern for *.tif files
folder_path = '../Lab_07/spatial_data/'  # change to your data folder path
pattern = '**/*.tif'

# Use glob.glob() with recursive=True to find files that match the pattern in all subdirectories
files = glob.glob(f'{folder_path}/{pattern}', recursive=True)

# Output the number of found files
print(f'Number of files with .tif extension: {len(files)}')


Number of files with .tif extension: 68


Create "resampled" directory

In [15]:
import os
import geopandas as gpd

# Read file fields_updated.geojson
fields = gpd.read_file("../Lab_07/fields_updated.geojson")

name_column = fields["Name"]

# Create directory "resampled"
os.makedirs("resampled", exist_ok=True)

# We cycle through each entry in the "Name" column and create directories in resampled
for name in name_column:
    entry_dir = os.path.join("resampled", name)
    os.makedirs(entry_dir, exist_ok=True)


In [22]:
import os
import geopandas as gpd
import rasterio as rastr
from rasterio.features import geometry_mask
from rasterio import mask as msk

# Read file field_polygons
fields = gpd.read_file("../Lab_07/fields_updated.geojson")

# Read file spatial_data
spatial_data = rastr.open("../Lab_07/spatial_data/201504022343_179D/29.566633136000007_49.434589403000004_31.185382877000006_50.486672568.tif")


for index, row in fields.iterrows():
    name = row["Name"] # Take name field
    geom = row["geometry"] # Take field geometry
    
    entry_dir = os.path.join("resampled", name)
    
    data, transform = rastr.mask.mask(spatial_data, [geom], crop=True)
    
    meta = spatial_data.meta.copy()
    
    # We update the metadata according to the extracted geometry
    meta.update({"driver": "GTiff",
                 "height": data.shape[1],
                 "width": data.shape[2],
                 "transform": transform})
    
    # Create the corresponding directory if it does not exist
    os.makedirs(entry_dir, exist_ok=True)
    
    # Write the raster file to the directory
    with rastr.open(os.path.join(entry_dir, "example.tif"), "w", **meta) as dst:
        dst.write(data)
